In [1]:
import numpy as np
from tensorflow.keras import layers, models, regularizers, optimizers, initializers
from tensorflow.keras.applications import VGG16

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from tensorflow.compat.v1.keras.backend import set_session
from tensorflow.compat.v1 import ConfigProto, Session
import tensorflow.keras
config = ConfigProto(allow_soft_placement=True)

config.gpu_options.per_process_gpu_memory_fraction = 0.7
set_session(Session(config=config))

In [2]:
conv_base = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(150, 150, 3)
)

In [3]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [4]:
from tensorflow.keras.preprocessing import image
train_datagen = image.ImageDataGenerator(
    1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
val_datagen = image.ImageDataGenerator(
    1.0/255
)

train_generator = train_datagen.flow_from_directory(
    '/home/oneran/dog-vs-cats/train',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

validation_generator = val_datagen.flow_from_directory(
    '/home/oneran/dog-vs-cats/val',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [5]:
def build_model():
    model = models.Sequential()
    model.add(
        conv_base
    )
    conv_base.trainable = False
    model.add(
        layers.Flatten()
    )
    # model.add(
    #     layers.Dropout(0.5)
    # )
    model.add(
        layers.Dense(512, activation='relu', kernel_initializer='he_normal')
    )
    model.add(
        layers.Dense(1, activation='sigmoid')
    )
    model.compile(
        optimizer=optimizers.Adagrad(),
        metrics=['acc'],
        loss='binary_crossentropy'
    )
    return model

In [6]:
from tensorflow.keras.callbacks import TensorBoard
import datetime

In [7]:
conv_base.trainable = False

In [8]:
log_dir = '/home/oneran/Mycodes/DeepLearning/log/VGG16_' + str(datetime.datetime.now()) + '_runtime_9'
callback_tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)
model = build_model()
model.fit_generator(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    callbacks=callback_tensorboard,
)

Epoch 1/30
100/100 [==============================] - 22s 169ms/step - loss: 3.8246 - acc: 0.7698 - val_loss: 1.0506 - val_acc: 0.9100
Epoch 2/30
100/100 [==============================] - 16s 160ms/step - loss: 1.3869 - acc: 0.8571 - val_loss: 0.7975 - val_acc: 0.9300
Epoch 3/30
100/100 [==============================] - 16s 160ms/step - loss: 0.9608 - acc: 0.8947 - val_loss: 0.6596 - val_acc: 0.9290
Epoch 4/30
  4/100 [>.............................] - ETA: 10s - loss: 0.4270 - acc: 0.9094

KeyboardInterrupt: 

In [9]:
test_datagen = image.ImageDataGenerator(
    1.0/255
)
test_generator = test_datagen.flow_from_directory(
    '/home/oneran/dog-vs-cats/test',
    class_mode='binary',
    target_size=(150, 150)    
)

Found 1000 images belonging to 2 classes.


In [40]:
model = models.load_model('/home/oneran/Documents/VGG16_ConvBase+Dense_512')

In [41]:
model.evaluate_generator(test_generator)

[0.34193533658981323, 0.9319999814033508]

In [42]:
model.get_layer('vgg16').trainable = True

In [43]:
model.get_layer('vgg16').summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [44]:
model.get_layer('vgg16').trainable = True
set_trainable = False
for layer in model.get_layer('vgg16').layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [45]:
model.get_layer('vgg16').summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

In [46]:
log_dir = '/home/oneran/Mycodes/DeepLearning/log/vgg16+ConvBlock5_NoFrozen_' + str(datetime.datetime.now()) + '_runtime_1'
callbacks_tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1)
model.compile(
    optimizer=optimizers.RMSprop(lr=1e-5),
    loss='binary_crossentropy',
    metrics=['acc']
)
model.fit_generator(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=callbacks_tensorboard
)

Epoch 1/50
100/100 [==============================] - 20s 192ms/step - loss: 0.1010 - acc: 0.9698 - val_loss: 0.3250 - val_acc: 0.9490
Epoch 2/50
100/100 [==============================] - 19s 185ms/step - loss: 0.1272 - acc: 0.9630 - val_loss: 0.2870 - val_acc: 0.9530
Epoch 3/50
100/100 [==============================] - 19s 186ms/step - loss: 0.0645 - acc: 0.9786 - val_loss: 0.3492 - val_acc: 0.9510
Epoch 4/50
100/100 [==============================] - 19s 186ms/step - loss: 0.0907 - acc: 0.9657 - val_loss: 0.3155 - val_acc: 0.9490
Epoch 5/50
100/100 [==============================] - 19s 186ms/step - loss: 0.1357 - acc: 0.9583 - val_loss: 0.3212 - val_acc: 0.9500
Epoch 6/50
100/100 [==============================] - 19s 187ms/step - loss: 0.1112 - acc: 0.9689 - val_loss: 0.3126 - val_acc: 0.9550
Epoch 7/50
100/100 [==============================] - 19s 187ms/step - loss: 0.1139 - acc: 0.9649 - val_loss: 0.2980 - val_acc: 0.9550
Epoch 8/50
100/100 [==============================] - 1

In [47]:
model.evaluate_generator(
    test_generator
)

[0.4288623332977295, 0.9520000219345093]